Apparent Competition

In [1]:
push!(LOAD_PATH, pwd()) # put current directory on path
using Revise
using UnPack
#include("ModelHelperFuns.jl")  # load the file
#using .ModelHelperFuns         # bring the module into scope
using CooperativeHuntingPkg
using Plots
default(
    guidefontsize=14,   # controls xlabel, ylabel, title font size
    tickfontsize=12,    # controls axis number labels
    legendfontsize=14,  # controls legend font
    linewidth=2,        # controls default line thickness
    grid = false,        # turns off grid in background
    fontfamily="Computer Modern" # font family that matches latex
)
using BifurcationKit, DifferentialEquations
# pgfplotsx()  # Set PGFPlotsX as the backend # this seems to not work on my computer, or on vscode.
using Measures # helps for adjusting location of axis labels


In [ ]:
# do vary scale case
# get stable coexistence equilibrium from do_continuation
using LinearAlgebra

function fun_Jac(N1, N2, gvec, params)
    x_max = params[:x_max]
    xvec = 1:x_max
    size = length(gvec) + 2

    # Gradients of functional responses
    grad_f_1 = fun_grad_func_response(1, xvec, N1, N2, params)
    grad_f_2 = fun_grad_func_response(2, xvec, N1, N2, params)

    Jac = zeros(size, size)
    Jac[1, :] .= fun_grad_big_prey(N1, N2, gvec, grad_f_1, params)
    Jac[2, :] .= fun_grad_small_prey(N1, N2, gvec, grad_f_2, params)
    Jac[3:end, :] .= fun_Jac_groups_helper(N1, N2, gvec, grad_f_1, grad_f_2, xvec, params)

    return Jac
end

function fun_Jac_groups(N1, N2, gvec, params)
    x_max = params[:x_max]
    xvec = 1:x_max

    grad_f_1 = zeros(2, x_max)
    grad_f_2 = zeros(2, x_max)

    Jac_need_to_trim = fun_Jac_groups_helper(N1, N2, gvec, grad_f_1, grad_f_2, xvec, params)
    Jac = Jac_need_to_trim[:, 3:end]
    return Jac
end

function classify_stability(J)
    eigenvalues = eigvals(J)
    real_parts = real(eigenvalues)

    if all(real_parts .< 0)
        return "Stable (attractive)"
    elseif any(real_parts .> 0)
        return "Unstable"
    elseif all(real_parts .<= 0)
        return "Marginally stable (needs further analysis)"
    else
        return "Indeterminate stability (needs further analysis)"
    end
end

function classify_equilibrium(equilibrium, params)
    N1, N2, gvec = equilibrium[1], equilibrium[2], equilibrium[3:end]
    J = fun_Jac(N1, N2, gvec, params)
    if !all(isfinite.(J))
        println(J)
        println(equilibrium)
        println(params)
    end
    stability = classify_stability(J)
    return stability
end


function get_apparent_competition(N1, N2, gvec, params)
    Jac_full = fun_Jac(N1, N2, gvec, params)
    Jac_apparent = Jac_full[2:end, 2:end]
    left_side_eqn = -Jac_full[2:end, 1]
    pdv_N1 = Jac_apparent \ left_side_eqn
    apparent_comp = pdv_N1[1]
    return apparent_comp
end

function get_apparent_competition2(N1, N2, gvec, params)
    Jac_full = fun_Jac(N1, N2, gvec, params)
    Jac_apparent = deleteat!(deleteat!(copy(Jac_full), 2, dims=1), 2, dims=2)
    left_side_eqn = -deleteat!(Jac_full[:, 2], 2)
    pdv_N2 = Jac_apparent \ left_side_eqn
    apparent_comp2 = pdv_N2[1]
    return apparent_comp2
end


function fun_grad_func_response(i, x, N1, N2, A1, A2, params)
    """
    The gradient of the (scaled) functional response on prey i wrt N1, N2.
    Returns a 2x(length(x)) matrix where rows correspond to N1 and N2.
    """
    alpha1 = fun_alpha1(x, params)
    alpha2 = fun_alpha2(x, params)
    H1 = fun_H1(x, params)
    H2 = fun_H2(x, params)
    denom = (1 .+ alpha1 .* H1 .* N1 .+ alpha2 .* N2 .* H2).^2

    if i == 1
        return [A1 .* alpha1 .* (1 .+ alpha2 .* H2 .* N2), 
                -A1 .* alpha1 .* alpha2 .* H2 .* N1] ./ denom
    elseif i == 2
        return [-A2 .* alpha1 .* alpha2 .* H1 .* N2,
                A2 .* alpha2 .* (1 .+ alpha1 .* H1 .* N1)] ./ denom
    else
        error("Invalid value for i. Must be 1 or 2.")
    end
end

function fun_grad_big_prey(N1, N2, gvec, grad_f_1, η1, x_max, params)
    """
    Return gradient of big prey wrt N1, N2, g(1), ..., g(x_max).
    """
    # The sum of g(x) * [∂f_1/∂N1, ∂f_1/∂N2]
    grad_sum_g_y = sum(grad_f_1 .* gvec, dims=2)

    delU1_N1 = η1 * (1 - 2 * N1) - grad_sum_g_y[1]
    delU1_N2 = -grad_sum_g_y[2]

    xvec = 1:x_max
    f1_vec = fun_response_non_dim(xvec, N1, N2, 1, params)
    delU1_g = -f1_vec

    return vcat([delU1_N1, delU1_N2], delU1_g)
end

function fun_grad_small_prey(N1, N2, gvec, grad_f_2, η2, x_max, params)
    """
    Return gradient of small prey wrt N1, N2, g(1), ..., g(x_max).
    """
    # The sum of g(x) * [∂f_2/∂N1, ∂f_2/∂N2]
    grad_sum_g_y = sum(grad_f_2 .* gvec, dims=2)

    delU2_N1 = -grad_sum_g_y[1]
    delU2_N2 = η2 * (1 - 2 * N2) - grad_sum_g_y[2]

    xvec = 1:x_max
    f2_vec = fun_response_non_dim(xvec, N1, N2, 2, params)
    delU2_g = -f2_vec

    return vcat([delU2_N1, delU2_N2], delU2_g)
end

function fun_Jac_groups_helper(N1, N2, gvec, grad_f_1, grad_f_2, xvec, x_max, Tx, d, η1, η2, params)
    """
    Calculates pdv{Q_i}{N_1}, pdv{Q_i}{N_2}, pdv{Q_i}{g(1)}... pdv{Q_i}{g(x_max)} 
    and stacks them into a matrix where Q_i is the right side of the equation for dg(i)/dT.
    """
    Jac = zeros(length(gvec), length(gvec) + 2)

    # Partial derivatives of yield (π) with respect to N1, N2
    partial_π = params[:β1] .* grad_f_1 .+ params[:β2] .* grad_f_2
    π_vec = yield_from_prey_non_dim(xvec, N1, N2, params)
    fitnessvec = π_vec ./ xvec

    # Partial derivative of S(1,x) with respect to N1, N2
    partial_S_vec = [fun_partial_S_wrt_prey(N1, N2, x, fitnessvec, π_vec, partial_π, d, params) for x in 2:x_max]
    S_x_1_vec = [best_response_fun_given_fitness(x, 1, fitnessvec, d) for x in 2:x_max]
    td = 1 - η1 - η2

    g(x) = gvec[x - 1]
    partial_S(x) = partial_S_vec[x - 2]
    S(x, y=1) = y == 1 ? S_x_1_vec[x - 2] : (x == 1 ? 1 - S_x_1_vec[y - 2] : error("Invalid S(x, y)"))
    π(x) = π_vec[x - 1]

    # First row
    Q1_Ni_group = (1 / Tx) * (2 * g(2) * partial_S(2) + sum([partial_S(x) .* (x * g(x) + g(1) * g(x - 1)) for x in 2:x_max]))
    Q1_Ni_pop = g(x_max) .* partial_π[:, end] - g(1) .* partial_π[:, 1]
    Q1_Ni = Q1_Ni_group .+ Q1_Ni_pop

    Q1_g1 = (-2 * g(1) * S(2) - sum([g(x - 1) * S(x, 1) for x in 3:x_max])) / Tx - π(1) - td
    Q1_g2 = (4 * S(1, 2) - g(1) * S(3, 1)) / Tx + 2 * td
    Q1_gx = [(x * S(1, x) - g(1) * S(x + 1, 1)) / Tx for x in 3:(x_max - 1)]
    Q1_gxmax = x_max * S(1, x_max) / Tx + π(x_max)
    Jac[1, :] .= vcat(Q1_Ni, Q1_g1, Q1_g2, Q1_gx..., Q1_gxmax)

    # Second row
    Q2_Ni = (1 / Tx) * (-partial_S(2) .* (2 * g(2) + 0.5 * g(1)^2) + partial_S(3) .* (3 * g(3) + g(1) * g(2))) +
             g(1) .* partial_π[:, 1] - g(2) .* partial_π[:, 2]
    Q2_g = zeros(length(gvec))
    Q2_g[1] = (1 / Tx) * (g(1) * S(2, 1) - g(2) * S(3, 1)) + π(1)
    Q2_g[2] = -(1 / Tx) * (2 * S(1, 2) + g(1) * S(3, 1)) - π(2) - 2 * td
    Q2_g[3] = (3 / Tx) * S(1, 3) + 3 * td
    Jac[2, :] .= vcat(Q2_Ni, Q2_g)

    # Rows for 2 < x < x_max
    for x in 3:(x_max - 1)
        Qx_Ni = (1 / Tx) * (partial_S(x + 1) .* ((x + 1) * g(x + 1) + g(1) * g(x)) -
                            partial_S(x) .* (x * g(x) + g(1) * g(x - 1))) +
                 g(x - 1) .* partial_π[:, x - 2] - g(x) .* partial_π[:, x - 1]

        Qx_g = zeros(length(gvec))
        Qx_g[1] = (1 / Tx) * (g(x - 1) * S(x, 1) - g(x) * S(x + 1, 1))
        Qx_g[x - 1] = (1 / Tx) * g(1) * S(x, 1) + π(x - 1)
        Qx_g[x] = -(1 / Tx) * (x * S(1, x) + g(1) * S(x + 1, 1)) - π(x) - x * td
        Qx_g[x + 1] = (1 / Tx) * (x + 1) * S(1, x + 1) + (x + 1) * td

        Jac[x, :] .= vcat(Qx_Ni, Qx_g)
    end

    # Last row
    Qxmax_Ni = -(1 / Tx) * partial_S(x_max) .* (x_max * g(x_max) + g(1) * g(x_max - 1)) +
                g(x_max - 1) .* partial_π[:, x_max - 2]
    Qxmax_g = zeros(length(gvec))
    Qxmax_g[1] = (1 / Tx) * g(x_max - 1) * S(x_max, 1)
    Qxmax_g[x_max - 1] = (1 / Tx) * g(1) * S(x_max, 1) + π(x_max - 1)
    Qxmax_g[x_max] = -(1 / Tx) * x_max * S(1, x_max) - x_max * td
    Jac[end, :] .= vcat(Qxmax_Ni, Qxmax_g)

    return Jac
end

function fun_partial_S_wrt_prey(N1, N2, x, π_vec, fitnessvec, partial_π, d, params)
    """
    Partial derivative of S(1,x) with respect to N1, N2 for a specified x.
    Returns an array with 2 entries corresponding to N1, N2.
    """
    S_1_x = best_response_fun_given_fitness(1, x, fitnessvec, d)
    S_x_1 = 1 - S_1_x

    partial_W_of_1 = partial_π[:, 1]
    partial_W_of_x = partial_π[:, x - 1] ./ x

    W_of_1 = π_vec[1]
    W_of_x = π_vec[x - 1] / (x - 1)

    partial_S_1_x = d * S_1_x * S_x_1 .* (partial_W_of_x - partial_W_of_1)
    return partial_S_1_x
end
